In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import time
from pandas.tseries.offsets import *
from scipy import stats
import statsmodels.api as sm
import datetime
import math

In [2]:
#1.1导入IPO数据
program_path = 'D:\\China A share\\'
Start_Date = '1997-01-01'
End_Date = '2019-06-30'
save_list = ['permno', 'ShortName', 'ListedDate', 'DelistedDate',
             'IPOIndustryCode', 'IPOIndustryName', 'LatestIndustryName',
             'LatestIndustryCode']

IPO = pd.read_excel(program_path + 'rawwind\\ER_IPO.xlsx', encoding='utf-16')
IPO = IPO.iloc[2:,:]
IPO['ListedDate'] = pd.to_datetime(IPO['ListedDate'], format='%Y/%m/%d')
IPO['DelistedDate'] = pd.to_datetime(IPO['DelistedDate'], format='%Y/%m/%d')
IPO['permno'] = IPO['Symbol'].copy()
IPO['Symbol'] = IPO['Symbol'].apply(lambda x: x[:3])
# 保留A股
IPO = IPO[
    (IPO['permno'].str.startswith('0')) | (IPO['permno'].str.startswith('3')) | (IPO['permno'].str.startswith('6'))]
# 剔除 688开头的科创板
IPO = IPO[IPO['Symbol'] != '688']

# 调整和剔除代码变更
IPO['permno'].replace('000022', '001872', inplace=True)
IPO['permno'].replace('601313', '601360', inplace=True)
IPO['permno'].replace('600087', '601975', inplace=True)
IPO.drop(index=IPO[IPO['permno'] == '600849'].index, inplace=True)

#为了IPO数据重复，发生二次上市的企业都用不同代码标注
# debug for 600018
temp_index = IPO[(IPO['permno'] == '600018') & (IPO['ListedDate'] == pd.datetime(2000, 7, 19))].index
IPO.loc[temp_index, 'permno'] = 'T00018'
#debug for 601975
temp_index = IPO[(IPO['permno'] == '601975') & (IPO['ListedDate'] == pd.datetime(1997, 6, 12))].index
IPO.loc[temp_index, 'permno'] = '600087'


IPO_sub = IPO.loc[:, save_list]

IPO_sub.to_pickle(program_path + 'temp\\IPO_list.pickle')
IPO_sub.to_csv(program_path + 'temp\\IPO_list.csv')

In [5]:
#1.2生成完美的list
#生成股票list和1997-2018每季度最后一天的时间序列表格
qlist=pd.DataFrame()
for k in range(2019-1997+1):
    qlist=np.append(qlist,pd.datetime(1997+k,3,31))
    qlist=np.append(qlist,pd.datetime(1997+k,6,30))
    qlist=np.append(qlist,pd.datetime(1997+k,9,30))
    qlist=np.append(qlist,pd.datetime(1997+k,12,31))
#concad，得到最长序列，根据IPO裁剪
#得到完美list，用reportdate汇整
qlist=qlist[qlist<pd.datetime(2019,8,1)]

In [6]:
permnolist=[]
for k in IPO_sub['permno'].values:
    q=pd.Series(qlist)
    permnolist.append(q)
q=pd.concat(permnolist,keys=IPO_sub['permno'].values).reset_index().drop('level_1',axis=1,inplace=False).rename(columns={'level_0':'permno',0:'REPORT_PERIOD'},inplace=False)
q=pd.merge(q,IPO_sub,on='permno',how='left')
q=q[(q['REPORT_PERIOD']>=q['ListedDate'])&((q['REPORT_PERIOD']<=q['DelistedDate'])|(q['DelistedDate'].isnull()))].loc[:,['permno','REPORT_PERIOD','ListedDate','DelistedDate']]

In [7]:
#在2002年以前只保留半年度报表
q=q[(q['REPORT_PERIOD']>pd.datetime(2002,1,1))|((q['REPORT_PERIOD'].map(lambda x:x.month==6))|(q['REPORT_PERIOD'].map(lambda x:x.month==12)))]

In [9]:
#2.2 检查现金流量表的完整
qcf=pd.merge(q,cf,on=['permno','REPORT_PERIOD'],how='left')
#查看缺失值：1997年后，未披露现金流量表，且没有在下一年4月以前退市
qcf[(qcf['ANN_DT'].isnull())&(qcf['REPORT_PERIOD']>pd.datetime(1997,12,31))&((qcf['REPORT_PERIOD']+MonthEnd(4)<=qcf['DelistedDate'])|qcf['DelistedDate'].isnull())]

,permno,REPORT_PERIOD,ListedDate,DelistedDate,OBJECT_ID,S_INFO_WINDCODE,WIND_CODE,ANN_DT,STATEMENT_TYPE,CRNCY_CODE,...,SPE_BAL_NETCASH_EQU_UNDIR,TOT_BAL_NETCASH_EQU_UNDIR,SPE_BAL_NETCASH_INC_UNDIR,TOT_BAL_NETCASH_INC_UNDIR,S_INFO_COMPCODE,S_DISMANTLE_CAPITAL_ADD_NET,IS_CALCULATION,SECURITIE_NETCASH_RECEIVED,OPDATE,OPMODE
10,000001,2002-03-31,1991-04-03,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,000001,2002-09-30,1991-04-03,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,000002,1998-06-30,1991-01-29,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,000002,2002-03-31,1991-01-29,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,000002,2002-09-30,1991-01-29,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,000003,1998-06-30,1991-07-03,2002-06-14,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,000003,1999-06-30,1991-07-03,2002-06-14,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,000004,1998-06-30,1991-01-14,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,000004,2002-03-31,1991-01-14,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,000004,2002-09-30,1991-01-14,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#缺失数据作图
totalcount=qcf.groupby(['REPORT_PERIOD'])['permno'].count().reset_index().rename(columns={'permno':'TOTAL'},inplace=False)
missingcount=qcf[(qcf['ANN_DT'].isnull())].groupby(['REPORT_PERIOD'])['permno'].count().reset_index().rename(columns={'permno':'Missing'},inplace=False)
missingdel=qcf[(qcf['ANN_DT'].isnull())&((qcf['REPORT_PERIOD']+MonthEnd(4)<=qcf['DelistedDate'])|qcf['DelistedDate'].isnull())].groupby(['REPORT_PERIOD'])['permno'].count().reset_index().rename(columns={'permno':'Missing nonDEL'},inplace=False)
qcf_count=pd.merge(totalcount,missingcount,on=['REPORT_PERIOD'],how='left')
qcf_count=pd.merge(qcf_count,missingdel,on=['REPORT_PERIOD'],how='left')
qcf_count.to_csv(program_path + 'tempwind\\qcf_count.csv')

In [11]:
#剔除缺失值中的2002年以前数据
qcf[(qcf['ANN_DT'].isnull())&
   (qcf['REPORT_PERIOD'].map(lambda x:x.year>2002))&
    (qcf['REPORT_PERIOD']>pd.datetime(1997,12,31))&((qcf['REPORT_PERIOD']+MonthEnd(4)<=qcf['DelistedDate'])|qcf['DelistedDate'].isnull())]

,permno,REPORT_PERIOD,ListedDate,DelistedDate,OBJECT_ID,S_INFO_WINDCODE,WIND_CODE,ANN_DT,STATEMENT_TYPE,CRNCY_CODE,...,SPE_BAL_NETCASH_EQU_UNDIR,TOT_BAL_NETCASH_EQU_UNDIR,SPE_BAL_NETCASH_INC_UNDIR,TOT_BAL_NETCASH_INC_UNDIR,S_INFO_COMPCODE,S_DISMANTLE_CAPITAL_ADD_NET,IS_CALCULATION,SECURITIE_NETCASH_RECEIVED,OPDATE,OPMODE
17454,000638,2003-03-31,1996-11-26,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20465,000693,2018-12-31,1997-02-26,2019-07-09,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22817,000730,2003-03-31,1997-05-22,2004-09-24,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65524,002680,2018-09-30,2012-06-05,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65525,002680,2018-12-31,2012-06-05,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65526,002680,2019-03-31,2012-06-05,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65527,002680,2019-06-30,2012-06-05,NaT,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86859,T00018,2003-03-31,2000-07-19,2006-10-20,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86860,T00018,2003-06-30,2000-07-19,2006-10-20,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86861,T00018,2003-09-30,2000-07-19,2006-10-20,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#3.2 生成现金流量表
lastqcf=qcf.drop(['OBJECT_ID',
       'S_INFO_WINDCODE', 'WIND_CODE', 'ANN_DT', 'STATEMENT_TYPE',
       'CRNCY_CODE','ACTUAL_ANN_DT','S_INFO_COMPCODE','IS_CALCULATION', 'OPDATE', 'OPMODE'],axis=1)
lastqcf.loc[lastqcf['REPORT_PERIOD']>=pd.datetime(2001,12,31),'reportdate']=lastqcf['REPORT_PERIOD']+MonthEnd(3)
lastqcf.loc[lastqcf['REPORT_PERIOD']<pd.datetime(2001,12,31),'reportdate']=lastqcf['REPORT_PERIOD']+MonthEnd(6)
lastqcf.rename(columns={'REPORT_PERIOD':'lastrepodate'},inplace=True)
lastqcf.rename(columns={'reportdate':'REPORT_PERIOD'},inplace=True)

In [13]:
#！！！！！！更正

qcfcol=qcf.columns.drop(['permno','REPORT_PERIOD','ListedDate','DelistedDate','OBJECT_ID',
       'S_INFO_WINDCODE', 'WIND_CODE', 'ANN_DT', 'STATEMENT_TYPE',
       'CRNCY_CODE','ACTUAL_ANN_DT','S_INFO_COMPCODE','IS_CALCULATION', 'OPDATE', 'OPMODE'],)
for k in qcfcol:
    lastqcf.rename(columns={k:k+'last'},inplace=True)
diffqcf=pd.merge(qcf,lastqcf,on=['permno','REPORT_PERIOD','ListedDate','DelistedDate'],how='left')
for k in qcfcol:
#！！！！！！！此处更正
    diffqcf.loc[(diffqcf['REPORT_PERIOD'].map(lambda x:x.month==3))&(diffqcf['REPORT_PERIOD'].map(lambda x:x.year>=2002)),k+'_diff']=diffqcf[k]
    diffqcf.loc[(diffqcf['REPORT_PERIOD'].map(lambda x:x.month!=3))&(diffqcf['REPORT_PERIOD'].map(lambda x:x.year>=2002)),k+'_diff']=diffqcf[k]-diffqcf[k+'last']
    diffqcf.loc[(diffqcf['REPORT_PERIOD'].map(lambda x:x.month==6))&(diffqcf['REPORT_PERIOD'].map(lambda x:x.year<2002)),k+'_diff']=diffqcf[k]
    diffqcf.loc[(diffqcf['REPORT_PERIOD'].map(lambda x:x.month!=6))&(diffqcf['REPORT_PERIOD'].map(lambda x:x.year<2002)),k+'_diff']=diffqcf[k]-diffqcf[k+'last']
    diffqcf.drop([k,k+'last'],axis=1,inplace=True)

In [58]:
#缺失数据作图
totalcount=diffqcf.groupby(['REPORT_PERIOD'])['permno'].count().reset_index().rename(columns={'permno':'TOTAL'},inplace=False)
missingcount=diffqcf[(diffqcf['ANN_DT'].isnull())].groupby(['REPORT_PERIOD'])['permno'].count().reset_index().rename(columns={'permno':'Missing'},inplace=False)
missingdel=diffqcf[(diffqcf['ANN_DT'].isnull())&((diffqcf['REPORT_PERIOD']+MonthEnd(4)<=diffqcf['DelistedDate'])|diffqcf['DelistedDate'].isnull())].groupby(['REPORT_PERIOD'])['permno'].count().reset_index().rename(columns={'permno':'Missing nonDEL'},inplace=False)
diffqcf_count=pd.merge(totalcount,missingcount,on=['REPORT_PERIOD'],how='left')
diffqcf_count=pd.merge(diffqcf_count,missingdel,on=['REPORT_PERIOD'],how='left')
diffqcf_count.to_csv(program_path + 'tempwind\\diffqcf_count.csv')

In [84]:
#FINAN=pd.read_excel(program_path + 'tempwind\\FINAN.xlsx')
#incomename=FINAN[FINAN['sheet']=='income']['names'].values
#balancename=FINAN[FINAN['sheet']=='balance']['names'].values
#cfname=FINAN[FINAN['sheet']=='cash flow']['names'].values


In [ ]:
#1.非数值信息
数据标签:
    ['S_INFO_WINDCODE', 'ANN_DT','ACTUAL_ANN_DT','STATEMENT_TYPE','REPORT_PERIOD']

损益表标签：
    ['NET_PROFIT_INCL_MIN_INT_INC', 'OPER_REV', 'LESS_OPER_COST',
       'OPER_PROFIT', 'INC_TAX', 'TOT_PROFIT', 'LESS_GERL_ADMIN_EXP',
       'LESS_SELLING_DIST_EXP', 'LESS_FIN_EXP', 'TOT_OPER_REV',
       'OTHER_BUS_INC', 'TOT_OPER_COST', 'RD_EXPENSE',
       'LESS_TAXES_SURCHARGES_OPS', 'PLUS_NON_OPER_REV',
       'LESS_NON_OPER_EXP', 'EBIT', 'EBITDA']

现金流量表标签：
    ['NET_CASH_FLOWS_OPER_ACT', 'NET_CASH_FLOWS_FNC_ACT',
       'NET_CASH_FLOWS_INV_ACT', 'FREE_CASH_FLOW',
       'NET_INCR_CASH_CASH_EQU', 'PAY_ALL_TYP_TAX', 'CASH_PAID_INVEST',
       'NET_CASH_PAY_AQUIS_SOBU', 'OTHER_CASH_PAY_RAL_INV_ACT',
       'CASH_PAY_ACQ_CONST_FIOLTA', 'DEPR_FA_COGA_DPBA']

资产负债表：
    ['TOT_SHRHLDR_EQY_INCL_MIN_INT', 'TOT_ASSETS', 'TOT_LIAB',
       'DVD_PAYABLE', 'INVENTORIES', 'TOT_CUR_LIAB', 'FIX_ASSETS',
       'ACCT_RCV', 'FIX_ASSETS', 'MONETARY_CAP', 'RCV_INVEST',
       'ST_BORROW', 'LT_BORROW', 'TOT_SHRHLDR_EQY_EXCL_MIN_INT',
       'MINORITY_INT', 'LT_BORROW', 'ST_BORROW', 'TOT_CUR_ASSETS',
       'TOT_CUR_LIAB', 'CASH_DEPOSITS_CENTRAL_BANK', 'FIX_ASSETS_DISP',
       'OTHER_EQUITY_TOOLS_P_SHR', 'R_AND_D_COSTS']
